## GANs

In [ ]:
%matplotlib inline
from fastai.gen_doc.nbdoc import *
from fastai.vision import * 
from fastai.vision.gan import *

GAN stands for [Generative Adversarial Nets](https://arxiv.org/pdf/1406.2661.pdf) and were invented by Ian Goodfellow. The concept is that we will train two models at the same time: a generator and a critic. The generator will try to make new images similar to the ones in our dataset, and the critic's job will try to classify real images from the fake ones the generator does. The generator returns images, the discriminator a feature map (it can be a single number depending on the input size). Usually the discriminator will be trained to return 0. everywhere for fake images and 1. everywhere for real ones.

This module contains all the necessary function to create a GAN.

We train them against each other in the sense that at each step (more or less), we:
1. Freeze the generator and train the discriminator for one step by:
  - getting one batch of true images (let's call that `real`)
  - generating one batch of fake images (let's call that `fake`)
  - have the discriminator evaluate each batch and compute a loss function from that; the important part is that it rewards positively the detection of real images and penalizes the fake ones
  - update the weights of the discriminator with the gradients of this loss
  
  
2. Freeze the discriminator and train the generator for one step by:
  - generating one batch of fake images
  - evaluate the discriminator on it
  - return a loss that rewards posisitivly the discriminator thinking those are real images; the important part is that it rewards positively the detection of real images and penalizes the fake ones
  - update the weights of the generator with the gradients of this loss

In [ ]:
show_doc(GANLearner)

<h2 id="GANLearner" class="doc_header"><code>class</code> <code>GANLearner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L209" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLearner-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>GANLearner</code>(**`data`**:[`DataBunch`](/basic_data.html#DataBunch), **`generator`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`critic`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`gen_loss_func`**:`LossFunction`, **`crit_loss_func`**:`LossFunction`, **`switcher`**:[`Callback`](/callback.html#Callback)=***`None`***, **`gen_first`**:`bool`=***`False`***, **`switch_eval`**:`bool`=***`True`***, **`show_img`**:`bool`=***`True`***, **`clip`**:`float`=***`None`***, **\*\*`learn_kwargs`**) :: [`Learner`](/basic_train.html#Learner)

<div class="collapse" id="GANLearner-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLearner-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>GANLearner</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

A [`Learner`](/basic_train.html#Learner) suitable for GANs.  

This is the general constructor to create a GAN, you might want to use one of the factory methods that are easier to use. Create a GAN from [`data`](/vision.data.html#vision.data), a `generator` and a `critic`. The [`data`](/vision.data.html#vision.data) should have the inputs the `generator` will expect and the images wanted as targets.

`gen_loss_func` is the loss function that will be applied to the `generator`. It takes three argument `fake_pred`, `target`, `output` and should return a rank 0 tensor. `output` is the result of the `generator` applied to the input (the xs of the batch), `target` is the ys of the batch and `fake_pred` is the result of the `discriminator` being given `output`. `output`and `target` can be used to add a specific loss to the GAN loss (pixel loss, feature loss) and for a good training of the gan, the loss should encourage `fake_pred` to be as close to 1 as possible (the `generator` is trained to fool the `critic`).

`crit_loss_func` is the loss function that will be applied to the `critic`. It takes two arguments `real_pred` and `fake_pred`. `real_pred` is the result of the `critic` on the target images (the ys of the batch) and `fake_pred` is the result of the `critic` applied on a batch of fake, generated byt the `generator` from the xs of the batch.

`switcher` is a [`Callback`](/callback.html#Callback) that should tell the GAN when to switch from critic to generator and vice versa. By default it does 5 iterations of the critic for 1 iteration of the generator. The model begins the training with the `generator` if `gen_first=True`. If `switch_eval=True`, the model that isn't trained is switched on eval mode (left in training mode otherwise, which means some statistics like the running mean in batchnorm layers are updated, or the dropouts are applied).

`clip` should be set to a certain value if one wants to clip the weights (see the [Wassertein GAN](https://arxiv.org/pdf/1701.07875.pdf) for instance).

If `show_img=True`, one image generated by the GAN is shown at the end of each epoch.

### Factory methods

In [ ]:
show_doc(GANLearner.from_learners)

<h4 id="GANLearner.from_learners" class="doc_header"><code>from_learners</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L222" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLearner-from_learners-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>from_learners</code>(**`learn_gen`**:[`Learner`](/basic_train.html#Learner), **`learn_crit`**:[`Learner`](/basic_train.html#Learner), **`switcher`**:[`Callback`](/callback.html#Callback)=***`None`***, **`weights_gen`**:`Point`=***`None`***, **\*\*`learn_kwargs`**)

<div class="collapse" id="GANLearner-from_learners-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLearner-from_learners-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>from_learners</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create a GAN from `learn_gen` and `learn_crit`.  

Directly creates a [`GANLearner`](/vision.gan.html#GANLearner) from two [`Learner`](/basic_train.html#Learner): one for the `generator` and one for the `critic`. The `switcher` and all `kwargs` will be passed to the initialization of [`GANLearner`](/vision.gan.html#GANLearner) along with the following loss functions:

- `loss_func_crit` is the mean of `learn_crit.loss_func` applied to `real_pred` and a target of ones with `learn_crit.loss_func` applied to `fake_pred` and a target of zeros
- `loss_func_gen` is the mean of `learn_crit.loss_func` applied to `fake_pred` and a target of ones (to full the discriminator) with `learn_gen.loss_func` applied to `output` and `target`. The weights of each of those contributions can be passed in `weights_gen` (default is 1. and 1.)

In [ ]:
show_doc(GANLearner.wgan)

<h4 id="GANLearner.wgan" class="doc_header"><code>wgan</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L229" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLearner-wgan-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>wgan</code>(**`data`**:[`DataBunch`](/basic_data.html#DataBunch), **`generator`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`critic`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`switcher`**:[`Callback`](/callback.html#Callback)=***`None`***, **`clip`**:`float`=***`0.01`***, **\*\*`learn_kwargs`**)

<div class="collapse" id="GANLearner-wgan-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLearner-wgan-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>wgan</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create a WGAN from `data`, `generator` and `critic`.  

The Wasserstein GAN is detailed in [this article]. `switcher` and the `kwargs` will be passed to the [`GANLearner`](/vision.gan.html#GANLearner) init, `clip`is the weight clipping.

## Switchers

In any GAN training, you will need to tell the [`Learner`](/basic_train.html#Learner) when to switch from generator to critic and vice versa. The two following [`Callback`](/callback.html#Callback) are examples to help you with that.

As usual, don't call the `on_something` methods directly, the fastai library will do it for you during training.

In [ ]:
show_doc(FixedGANSwitcher, title_level=3)

<h3 id="FixedGANSwitcher" class="doc_header"><code>class</code> <code>FixedGANSwitcher</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L156" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#FixedGANSwitcher-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>FixedGANSwitcher</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`n_crit`**:`Union`\[`int`, `Callable`\]=***`1`***, **`n_gen`**:`Union`\[`int`, `Callable`\]=***`1`***) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

<div class="collapse" id="FixedGANSwitcher-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#FixedGANSwitcher-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>FixedGANSwitcher</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Switcher to do `n_crit` iterations of the critic then `n_gen` iterations of the generator.  

In [ ]:
show_doc(FixedGANSwitcher.on_train_begin)

<h4 id="FixedGANSwitcher.on_train_begin" class="doc_header"><code>on_train_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L162" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#FixedGANSwitcher-on_train_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_train_begin</code>(**\*\*`kwargs`**)

<div class="collapse" id="FixedGANSwitcher-on_train_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#FixedGANSwitcher-on_train_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_train_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Initiate the iteration counts.  

In [ ]:
show_doc(FixedGANSwitcher.on_batch_end)

<h4 id="FixedGANSwitcher.on_batch_end" class="doc_header"><code>on_batch_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L166" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#FixedGANSwitcher-on_batch_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_batch_end</code>(**`iteration`**, **\*\*`kwargs`**)

<div class="collapse" id="FixedGANSwitcher-on_batch_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#FixedGANSwitcher-on_batch_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_batch_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Switch the model if necessary.  

In [ ]:
show_doc(AdaptiveGANSwitcher, title_level=3)

<h3 id="AdaptiveGANSwitcher" class="doc_header"><code>class</code> <code>AdaptiveGANSwitcher</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L180" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveGANSwitcher-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>AdaptiveGANSwitcher</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`gen_thresh`**:`float`=***`None`***, **`critic_thresh`**:`float`=***`None`***) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

<div class="collapse" id="AdaptiveGANSwitcher-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveGANSwitcher-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>AdaptiveGANSwitcher</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Switcher that goes back to generator/critic when the loss goes below `gen_thresh`/`crit_thresh`.  

In [ ]:
show_doc(AdaptiveGANSwitcher.on_batch_end)

<h4 id="AdaptiveGANSwitcher.on_batch_end" class="doc_header"><code>on_batch_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L186" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveGANSwitcher-on_batch_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_batch_end</code>(**`last_loss`**, **\*\*`kwargs`**)

<div class="collapse" id="AdaptiveGANSwitcher-on_batch_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveGANSwitcher-on_batch_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_batch_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Switch the model if necessary.  

## Discriminative LR

If you want to train your critic at a different learning rate than the generator, this will let you do it automatically (even if you have a learning rate schedule).

In [ ]:
show_doc(GANDiscriminativeLR, title_level=3)

<h3 id="GANDiscriminativeLR" class="doc_header"><code>class</code> <code>GANDiscriminativeLR</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L284" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANDiscriminativeLR-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>GANDiscriminativeLR</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`mult_lr`**:`float`=***`5.0`***) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

<div class="collapse" id="GANDiscriminativeLR-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANDiscriminativeLR-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>GANDiscriminativeLR</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

[`Callback`](/callback.html#Callback) that handles multiplying the learning rate by `mult_lr` for the critic.  

In [ ]:
show_doc(GANDiscriminativeLR.on_batch_begin)

<h4 id="GANDiscriminativeLR.on_batch_begin" class="doc_header"><code>on_batch_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L290" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANDiscriminativeLR-on_batch_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_batch_begin</code>(**`train`**, **\*\*`kwargs`**)

<div class="collapse" id="GANDiscriminativeLR-on_batch_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANDiscriminativeLR-on_batch_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_batch_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Multiply the current lr if necessary.  

In [ ]:
show_doc(GANDiscriminativeLR.on_step_end)

<h4 id="GANDiscriminativeLR.on_step_end" class="doc_header"><code>on_step_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L294" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANDiscriminativeLR-on_step_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_step_end</code>(**\*\*`kwargs`**)

<div class="collapse" id="GANDiscriminativeLR-on_step_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANDiscriminativeLR-on_step_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_step_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Put the LR back to its value if necessary.  

## Specific models

In [ ]:
show_doc(basic_critic)

<h4 id="basic_critic" class="doc_header"><code>basic_critic</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L17" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#basic_critic-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>basic_critic</code>(**`in_size`**:`int`, **`n_channels`**:`int`, **`n_features`**:`int`=***`64`***, **`n_extra_layers`**:`int`=***`0`***, **\*\*`conv_kwargs`**)

<div class="collapse" id="basic_critic-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#basic_critic-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>basic_critic</code>:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_basic_critic</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L55" class="source_link" style="float:right">[source]</a></li></ul><p>Some other tests where <code>basic_critic</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_module</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L66" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

A basic critic for images `n_channels` x `in_size` x `in_size`.  

This model contains a first 4 by 4 convolutional layer of stride 2 from `n_channels` to `n_features` followed by `n_extra_layers` 3 by 3 convolutional layer of stride 1. Then we put as many 4 by 4 convolutional layer of stride 2 with a number of features multiplied by 2 at each stage so that the `in_size` becomes 1. `kwargs` can be used to customize the convolutional layers and are passed to [`conv_layer`](/layers.html#conv_layer).

In [ ]:
show_doc(basic_generator)

<h4 id="basic_generator" class="doc_header"><code>basic_generator</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L28" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#basic_generator-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>basic_generator</code>(**`in_size`**:`int`, **`n_channels`**:`int`, **`noise_sz`**:`int`=***`100`***, **`n_features`**:`int`=***`64`***, **`n_extra_layers`**=***`0`***, **\*\*`conv_kwargs`**)

<div class="collapse" id="basic_generator-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#basic_generator-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>basic_generator</code>:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_basic_generator</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L45" class="source_link" style="float:right">[source]</a></li></ul><p>Some other tests where <code>basic_generator</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_module</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L66" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

A basic generator from `noise_sz` to images `n_channels` x `in_size` x `in_size`.  

This model contains a first 4 by 4 transposed convolutional layer of stride 1 from `noise_size` to the last numbers of features of the corresponding critic. Then we put as many 4 by 4 transposed convolutional layer of stride 2 with a number of features divided by 2 at each stage so that the image ends up being of height and widht `in_size//2`. At the end, we add`n_extra_layers` 3 by 3 convolutional layer of stride 1. The last layer is a transpose convolution of size 4 by 4 and stride 2 followed by `tanh`. `kwargs` can be used to customize the convolutional layers and are passed to [`conv_layer`](/layers.html#conv_layer).

In [ ]:
show_doc(gan_critic)

<h4 id="gan_critic" class="doc_header"><code>gan_critic</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L267" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#gan_critic-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>gan_critic</code>(**`n_channels`**:`int`=***`3`***, **`nf`**:`int`=***`128`***, **`n_blocks`**:`int`=***`3`***, **`p`**:`int`=***`0.15`***)

<div class="collapse" id="gan_critic-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#gan_critic-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>gan_critic</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Critic to train a `GAN`.  

In [ ]:
show_doc(GANTrainer)

<h2 id="GANTrainer" class="doc_header"><code>class</code> <code>GANTrainer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L71" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>GANTrainer</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`switch_eval`**:`bool`=***`False`***, **`clip`**:`float`=***`None`***, **`beta`**:`float`=***`0.98`***, **`gen_first`**:`bool`=***`False`***, **`show_img`**:`bool`=***`True`***) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

<div class="collapse" id="GANTrainer-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>GANTrainer</code>:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_trainer</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L80" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Handles GAN Training.  

[`LearnerCallback`](/basic_train.html#LearnerCallback) that will be responsible to handle the two different optimizers (one for the generator and one for the critic), and do all the work behind the scenes so that the generator (or the critic) are in training mode with parameters requirement gradients each time we switch.

`switch_eval=True` means that the [`GANTrainer`](/vision.gan.html#GANTrainer) will put the model that isn't training into eval mode (if it's `False` its running statistics like in batchnorm layers will be updated and dropout will be applied). `clip` is the clipping applied to the weights (if not `None`). `beta` is the coefficient for the moving averages as the [`GANTrainer`](/vision.gan.html#GANTrainer)tracks separately the generator loss and the critic loss. `gen_first=True` means the training begins with the generator (with the critic if it's `False`). If `show_img=True` we show a generated image at the end of each epoch.

In [ ]:
show_doc(GANTrainer.switch)

<h4 id="GANTrainer.switch" class="doc_header"><code>switch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L148" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-switch-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>switch</code>(**`gen_mode`**:`bool`=***`None`***)

<div class="collapse" id="GANTrainer-switch-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-switch-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>switch</code>:</p><p>Some other tests where <code>switch</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_module</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L66" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Switch the model, if `gen_mode` is provided, in the desired mode.  

If `gen_mode` is left as `None`, just put the model in the other mode (critic if it was in generator mode and vice versa).

In [ ]:
show_doc(GANTrainer.on_train_begin)

<h4 id="GANTrainer.on_train_begin" class="doc_header"><code>on_train_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L89" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_train_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_train_begin</code>(**\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_train_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_train_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_train_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create the optimizers for the generator and critic if necessary, initialize smootheners.  

In [ ]:
show_doc(GANTrainer.on_epoch_begin)

<h4 id="GANTrainer.on_epoch_begin" class="doc_header"><code>on_epoch_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L131" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_epoch_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_epoch_begin</code>(**`epoch`**, **\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_epoch_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_epoch_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_epoch_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Put the critic or the generator back to eval if necessary.  

In [ ]:
show_doc(GANTrainer.on_batch_begin)

<h4 id="GANTrainer.on_batch_begin" class="doc_header"><code>on_batch_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L109" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_batch_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_batch_begin</code>(**`last_input`**, **`last_target`**, **\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_batch_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_batch_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_batch_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Clamp the weights with `self.clip` if it's not None, return the correct input.  

In [ ]:
show_doc(GANTrainer.on_backward_begin)

<h4 id="GANTrainer.on_backward_begin" class="doc_header"><code>on_backward_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L116" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_backward_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_backward_begin</code>(**`last_loss`**, **`last_output`**, **\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_backward_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_backward_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_backward_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Record `last_loss` in the proper list.  

In [ ]:
show_doc(GANTrainer.on_epoch_end)

<h4 id="GANTrainer.on_epoch_end" class="doc_header"><code>on_epoch_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L135" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_epoch_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_epoch_end</code>(**`pbar`**, **`epoch`**, **`last_metrics`**, **\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_epoch_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_epoch_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_epoch_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Put the various losses in the recorder and show a sample image.  

In [ ]:
show_doc(GANTrainer.on_train_end)

<h4 id="GANTrainer.on_train_end" class="doc_header"><code>on_train_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L105" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_train_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_train_end</code>(**\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_train_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_train_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_train_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Switch in generator mode for showing results.  

## Specific modules

In [ ]:
show_doc(GANModule, title_level=3)

<h3 id="GANModule" class="doc_header"><code>class</code> <code>GANModule</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L41" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANModule-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>GANModule</code>(**`generator`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=***`None`***, **`critic`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=***`None`***, **`gen_mode`**:`bool`=***`False`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="GANModule-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANModule-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>GANModule</code>:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_module</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L66" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Wrapper around a `generator` and a `critic` to create a GAN.  

If `gen_mode` is left as `None`, just put the model in the other mode (critic if it was in generator mode and vice versa).

In [ ]:
show_doc(GANModule.switch)

<h4 id="GANModule.switch" class="doc_header"><code>switch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L50" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANModule-switch-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>switch</code>(**`gen_mode`**:`bool`=***`None`***)

<div class="collapse" id="GANModule-switch-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANModule-switch-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>switch</code>:</p><p>Some other tests where <code>switch</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_module</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L66" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Put the model in generator mode if `gen_mode`, in critic mode otherwise.  

In [ ]:
show_doc(GANLoss, title_level=3)

<h3 id="GANLoss" class="doc_header"><code>class</code> <code>GANLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L54" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLoss-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>GANLoss</code>(**`loss_funcG`**:`Callable`, **`loss_funcC`**:`Callable`, **`gan_model`**:[`GANModule`](/vision.gan.html#GANModule)) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`GANModule`](/vision.gan.html#GANModule)

<div class="collapse" id="GANLoss-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLoss-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>GANLoss</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Wrapper around `loss_funcC` (for the critic) and `loss_funcG` (for the generator).  

In [ ]:
show_doc(AdaptiveLoss, title_level=3)

<h3 id="AdaptiveLoss" class="doc_header"><code>class</code> <code>AdaptiveLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L298" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveLoss-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>AdaptiveLoss</code>(**`crit`**) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="AdaptiveLoss-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveLoss-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>AdaptiveLoss</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Expand the `target` to match the `output` size before applying `crit`.  

In [ ]:
show_doc(accuracy_thresh_expand)

<h4 id="accuracy_thresh_expand" class="doc_header"><code>accuracy_thresh_expand</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L306" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#accuracy_thresh_expand-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>accuracy_thresh_expand</code>(**`y_pred`**:`Tensor`, **`y_true`**:`Tensor`, **`thresh`**:`float`=***`0.5`***, **`sigmoid`**:`bool`=***`True`***) → `Rank0Tensor`

<div class="collapse" id="accuracy_thresh_expand-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#accuracy_thresh_expand-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>accuracy_thresh_expand</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Compute accuracy after expanding `y_true` to the size of `y_pred`.  

## Data Block API

In [ ]:
show_doc(NoisyItem, title_level=3)

<h3 id="NoisyItem" class="doc_header"><code>class</code> <code>NoisyItem</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L234" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#NoisyItem-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>NoisyItem</code>(**`noise_sz`**) :: [`ItemBase`](/core.html#ItemBase)

<div class="collapse" id="NoisyItem-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#NoisyItem-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>NoisyItem</code>:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_noisy_item</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L37" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

An random [`ItemBase`](/core.html#ItemBase) of size `noise_sz`.  

In [ ]:
show_doc(GANItemList, title_level=3)

<h3 id="GANItemList" class="doc_header"><code>class</code> <code>GANItemList</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L242" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANItemList-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>GANItemList</code>(**`items`**, **`noise_sz`**:`int`=***`100`***, **\*\*`kwargs`**) :: [`ImageList`](/vision.data.html#ImageList)

<div class="collapse" id="GANItemList-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANItemList-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>GANItemList</code>:</p><p>Some other tests where <code>GANItemList</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_datasets</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L30" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

`ItemList` suitable for GANs.  

Inputs will be [`NoisyItem`](/vision.gan.html#NoisyItem) of `noise_sz` while the default class for target is [`ImageList`](/vision.data.html#ImageList).

In [ ]:
show_doc(GANItemList.show_xys)

<h4 id="GANItemList.show_xys" class="doc_header"><code>show_xys</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L254" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANItemList-show_xys-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>show_xys</code>(**`xs`**, **`ys`**, **`imgsize`**:`int`=***`4`***, **`figsize`**:`Optional`\[`Tuple`\[`int`, `int`\]\]=***`None`***, **\*\*`kwargs`**)

<div class="collapse" id="GANItemList-show_xys-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANItemList-show_xys-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>show_xys</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Shows `ys` (target images) on a figure of `figsize`.  

In [ ]:
show_doc(GANItemList.show_xyzs)

<h4 id="GANItemList.show_xyzs" class="doc_header"><code>show_xyzs</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L258" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANItemList-show_xyzs-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>show_xyzs</code>(**`xs`**, **`ys`**, **`zs`**, **`imgsize`**:`int`=***`4`***, **`figsize`**:`Optional`\[`Tuple`\[`int`, `int`\]\]=***`None`***, **\*\*`kwargs`**)

<div class="collapse" id="GANItemList-show_xyzs-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANItemList-show_xyzs-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>show_xyzs</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Shows `zs` (generated images) on a figure of `figsize`.  

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(GANLoss.critic)

<h4 id="GANLoss.critic" class="doc_header"><code>critic</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L65" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLoss-critic-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>critic</code>(**`real_pred`**, **`input`**)

<div class="collapse" id="GANLoss-critic-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLoss-critic-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>critic</code>:</p><p>Some other tests where <code>critic</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_basic_critic</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L55" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Create some `fake_pred` with the generator from `input` and compare them to `real_pred` in `self.loss_funcD`.  

In [ ]:
show_doc(GANModule.forward)

<h4 id="GANModule.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L47" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANModule-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**\*`args`**)

<div class="collapse" id="GANModule-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANModule-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(GANLoss.generator)

<h4 id="GANLoss.generator" class="doc_header"><code>generator</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L60" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLoss-generator-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>generator</code>(**`output`**, **`target`**)

<div class="collapse" id="GANLoss-generator-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLoss-generator-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>generator</code>:</p><p>Some other tests where <code>generator</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_basic_generator</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L45" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Evaluate the `output` with the critic then uses `self.loss_funcG` to combine it with `target`.  

In [ ]:
show_doc(NoisyItem.apply_tfms)

<h4 id="NoisyItem.apply_tfms" class="doc_header"><code>apply_tfms</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L238" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#NoisyItem-apply_tfms-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>apply_tfms</code>(**`tfms`**, **\*\*`kwargs`**)

<div class="collapse" id="NoisyItem-apply_tfms-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#NoisyItem-apply_tfms-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>apply_tfms</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Subclass this method if you want to apply data augmentation with `tfms` to this [`ItemBase`](/core.html#ItemBase).  

In [ ]:
show_doc(AdaptiveLoss.forward)

<h4 id="AdaptiveLoss.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L303" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveLoss-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`output`**, **`target`**)

<div class="collapse" id="AdaptiveLoss-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveLoss-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(GANItemList.get)

<h4 id="GANItemList.get" class="doc_header"><code>get</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L251" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANItemList-get-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>get</code>(**`i`**)

<div class="collapse" id="GANItemList-get-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANItemList-get-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>get</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Subclass if you want to customize how to create item `i` from `self.items`.  

In [ ]:
show_doc(GANItemList.reconstruct)

<h4 id="GANItemList.reconstruct" class="doc_header"><code>reconstruct</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L252" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANItemList-reconstruct-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>reconstruct</code>(**`t`**)

<div class="collapse" id="GANItemList-reconstruct-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANItemList-reconstruct-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>reconstruct</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Reconstruct one of the underlying item for its data `t`.  

In [ ]:
show_doc(AdaptiveLoss.forward)

<h4 id="AdaptiveLoss.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L303" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveLoss-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`output`**, **`target`**)

<div class="collapse" id="AdaptiveLoss-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveLoss-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

## New Methods - Please document or move to the undocumented section